In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## **Technical Summary**


## **Business Understanding**
Cyclops Hospital Network (CHN) owns 4 inpatient hospitals and 27 urgent care centers. The 4 inpatient hospitals each have a pediatric emergency room. The 27 urgent care centers are equipped to perform X-ray and CT imaging and diagnosis/treat peditric patients as well. Overall, CHN thus has 31 locations where pediatric patients who potentially have viral or bacterial pneumonia may seek diagnosis and care. Given certain symptoms and the severity of those symptoms, many of those patients will undergo X-ray imaging of the thorax. Given that a radiologist has a maximum of 12 hours to review these images and the initial assessment of the imaging is done by either an emergency room physician or an urgent care practitioner, who may be less accurate in diagnosing pneumonia via imaging, CHN wishes to create a decision support tool (DST) using a neural network in order to check the assessment of the emergency room and urgent care physicians. 

This DST will help to prevent doctors from missing important diagnoses and sending patients home with lack of care. Given the wide range of timescales during which pneumonia can develop, the similarly wide range of severity of symptoms and the specific dillema many pediatric patients experience in verbalizing their symptoms/health status, this DST will protect at-risk patients from being sent home without care to potentially worsen before a radiologist reviews her or his imaging. Additionally, this DST will protect CHN from malpractice suits that could result from this lack of diagnosis and subsequent lack of care.

## Directory Setup
The purpose of this section will demostrate and justify our data preperations steps used on our data. In our preperation we set up generators for our data, and in the generators we also re-scale and re-size the images before they are fed into our model.

Due to the large size of our dataset, we chose to use Kaggle as a place to store our data on the cloud. Our directory setup was structured in the following format: 
```
├── chest-xray                    <- Top level directory
│   ├── test                      <- Test set images
│   │   ├── Normal                <- Normal lung photos      
│   │   │   └── ...
│   │   └──  Pneumonia            <- Pneumonia lung photos
│   │   │   └── ...
│   ├── train                     <- Training set images
│   │   ├── Normal                <- Normal lung photos 
│   │   │   └── ...
│   │   └──  Pneumonia            <- Pneumonia lung photos
│   │   │   └── ...
│   ├── val                       <- Training set images
│   │   ├── Normal                <- Normal lung photos 
│   │   │   └── ...
│   │   └──  Pneumonia            <- Pneumonia lung photos
│   │   │   └── ...             
```
This setup allows us to use Kera's Image Data Generator to load our data. We chose to use a generator for 3 reasons:
- Saving memory and disk space by not downloading the dataset
- Integrating the preproccesing into our modeling process
- Easy re-sizing and rescaling of images
Using generators also allows more easily reproducable results. Since images fed into our model this way do not have to be preproccessed beforehand.

In [ ]:
# Import statements
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import seaborn as sns

In [ ]:
# Instantiating a generator object and normalizing the RGB values
traingen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
testgen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
valgen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

# Creating the generator for the training data
train_data = traingen.flow_from_directory(
    # Specifying location of training data
    directory='../input/chest-xray-pneumonia/chest_xray/train',
    # Re-sizing images to 150x150
    target_size=(150, 150),
    # Class mode to binary to recoginize the two directories "NORMAL" and "PNEUMONIA" as the labels
    class_mode='binary',
    batch_size=20,
    seed=42
)
# Creating the generator for the testing data
test_data = testgen.flow_from_directory(
    # Specifying location of testing data
    directory='../input/chest-xray-pneumonia/chest_xray/test',
    # Re-sizing images to 150x150
    target_size=(150, 150),
    # Class mode to binary to recoginize the two directories "NORMAL" and "PNEUMONIA" as the labels
    class_mode='binary',
    batch_size=20,
    seed=42
)

# Setting aside a validation set
val_data = valgen.flow_from_directory(
    # Specifying location of testing data
    directory='../input/chest-xray-pneumonia/chest_xray/val',
    # Re-sizing images to 150x150
    target_size=(150, 150),
    # Class mode to binary to recoginize the two directories "NORMAL" and "PNEUMONIA" as the labels
    class_mode='binary',
    batch_size=20,
    seed=42
)

## **Data Understanding**
- Because we want to train a neural network to help identify whether or not a subject has pneumonia or not based on a chest X-ray, this dataset of 5,232 chest X-rays from children will help us train the network and so that it can be of use to doctors. There are 3,883 pneumonia x-rays and 1,349 normal ones, so there is a class imbalance issue. Additionally, each image is a different size, so it is necessary to standardize the images before modelling. 
- In the context of this data, a false positive would mean that the neural network identifies an x-ray as showing evidence of pneumonia, when it is really a normal x-ray. A false negative would mean that the neural network identifies a pneumonia image as being normal.

In [ ]:
# Putting class information into a dataframe for easy visualizing
classes = pd.DataFrame(train_data.classes)
values = classes.value_counts()


In [ ]:
# Giving the data better labels for visualization
class_dict = {0:'Normal', 1:'Pneumonia'}
classes[0] = classes[0].map(class_dict)


In [ ]:
# Looking at the distribution of values between x-rays that show pneumonia and those that don't
diag = classes[0].value_counts()
diag

In [ ]:
# Creating a bargraph to visualize the class imbalance
plt.figure(figsize=(12,8))
sns.set(font_scale=1.4)
sns.barplot(diag.index, diag.values)
plt.ylabel("Number of Images")
plt.title('Distribution of Chest X-ray Images');

**Visualize Transformation**

We will visualize the first 10 items in the training data set to check that all transformations to the images were done correctly.

In [ ]:
# Visualize
train_batch = train_data.next()
fig, axes = plt.subplots(2, 5, figsize=(16, 8))
    
for i in range(10):
    # Load image into numpy array and re-scale
    img = np.array(train_batch[0][i] * 255, dtype='uint8')
    ax = axes[i // 5, i % 5]
    ax.imshow(img)
fig.suptitle('Training Images')
plt.tight_layout()
plt.show()

## **Multilayer Perceptron Models**

We will start by setting up a baseline multi-layer perceptron model, because this is the most simple kind of Neural Network. We will iterate off of the results, building adjusted models with the goal of obtaining an improved model each time.

### **Baseline Multilayer Perceptron Model**

In [ ]:
# Setting up a baseline MLP
baseline = keras.Sequential(
    [
        keras.Input(shape=(150,150,3)), # Don't always need this input separately
        keras.layers.Flatten(), # need to flatten our images to be one long array
        keras.layers.Dense(100, activation="relu"),
        keras.layers.Dense(1, activation="sigmoid"),
    ])

baseline.summary()

In [ ]:
# Compiling the baseline MLP
baseline.compile(optimizer='adam',
                      loss='binary_crossentropy',
                      metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

In [ ]:
baseline_results = baseline.fit_generator(train_data,
                              steps_per_epoch=100,
                              epochs=10,
                              validation_data=test_data,)


In [ ]:
# Visualizing results
# Creating figure with 2 subplots
def visualize_results(results):
    
    fig, (ax1, ax2) = plt.subplots(1,2,figsize=(16, 8))
    # Geting training history from results
    history = results.history
    # Ploting on first subplot
    ax1.plot(history['loss'])
    ax1.plot(history['val_loss'])
    # Labeling 
    ax1.xaxis.set_label('Epochs')
    ax1.yaxis.set_label('Loss')
    ax1.legend(['loss', 'val_loss'])
    # Ploting on second subplot
    ax2.plot(history['acc'])
    ax2.plot(history['val_acc'])
    # Labeling 
    ax1.xaxis.set_label('Epochs')
    ax1.yaxis.set_label('Accuracy')
    ax2.legend(['Accuracy', 'Val_acc'])
    fig.suptitle('Loss and Accuracy of Model')

In [ ]:
def oes_matrix(results):
    """
    Plots a confusion matrix using the results atrribute of a Keras history object
  
    Parameters:
    results (keras.callbacks.History): 
  
    Returns:
    None
  
    """
    conf = np.array([[results.history['true_negatives'][-1], results.history['false_negatives'][-1]], [results.history['false_positives'][-1], results.history['true_positives'][-1]]])
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(conf.astype('int'), annot=True, fmt='g', ax=ax )

In [ ]:
visualize_results(baseline_results)

In [ ]:
oes_matrix(baseline_results)

**Conclusion**

This model is overfitting; training accuracy is around 95%, while testing data is at around 80%. Additionally, loss for testing data is oscillating too much.

### **Adding another layer to the baseline MLP**

In [ ]:
# Setting up a two layer MLP
two_hidden = keras.Sequential(
    [
        keras.Input(shape=(150,150,3)), # Don't always need this input separately
        keras.layers.Flatten(), # need to flatten our images to be one long array
        keras.layers.Dense(100, activation="relu"),
        keras.layers.Dense(50, activation="relu"),
        keras.layers.Dense(1, activation="sigmoid"),
    ])

two_hidden.summary()

In [ ]:
# Compiling the two hidden layer MLP
two_hidden.compile(optimizer='adam',
                      loss='binary_crossentropy',
                      metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

In [ ]:
# Fitting the two hidden layer MLP
two_hidden_results = two_hidden.fit_generator(train_data,
                              steps_per_epoch=100,
                              epochs=10,
                              validation_data=test_data)

In [ ]:
visualize_results(two_hidden_results)

**Conclusion**

Once again, the model is overfit. Also, judging from this graph showing the accuracy and loss for both training and validation data, it looks like it could benefit from a greater number of training epochs, since it looks like the validation loss might decrease further.

### **Adding Dropout layers to the two layer MLP**

In [ ]:
two_hidden_dropout = keras.Sequential(
    [
        keras.Input(shape=(150,150,3)), # Don't always need this input separately
        keras.layers.Flatten(), # need to flatten our images to be one long array
        keras.layers.Dense(100, activation="relu"),
        keras.layers.Dropout(0.25),
        keras.layers.Dense(50, activation="relu"),
        keras.layers.Dropout(0.25),
        keras.layers.Dense(1, activation="sigmoid"),
    ])

two_hidden_dropout.summary()

In [ ]:
# Compiling the two hidden layer dropout MLP
two_hidden_dropout.compile(optimizer='adam',
                      loss='binary_crossentropy',
                      metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

In [ ]:
# Fitting the two hidden layer dropout MLP
two_hidden_dropout_results = two_hidden_dropout.fit_generator(train_data,
                              steps_per_epoch=100,
                              epochs=10,
                              validation_data=test_data)

In [ ]:
visualize_results(two_hidden_dropout_results)

**Conclusion**

This model requires improvement; test accuracy does not improve at all over the epochs, and testing loss is oscillating too frequently and with too great a magnitude.

## **Convolutional Neural Network (CNN) Model Iterations**

The MLP models we have run so far are a good start, but in order to get a network which really capture all of the detail of the images and can pick up on patterns, we need to run some CNNs.

In [ ]:
# Create model
deep_cnn = keras.Sequential()

# Adding first Conv2D and MaxPool layer, starting small and then growing larger.
deep_cnn.add(keras.layers.Conv2D(32, (2, 2), activation='relu', input_shape=(150, 150, 3)))
deep_cnn.add(keras.layers.MaxPool2D(2, 2))

# Second layer with 64 filters
deep_cnn.add(keras.layers.Conv2D(64, (2, 2), activation='relu'))
deep_cnn.add(keras.layers.MaxPool2D(2, 2))

# Third layer with 96 filters
deep_cnn.add(keras.layers.Conv2D(96, (2, 2), activation='relu'))
deep_cnn.add(keras.layers.MaxPool2D(2, 2))
# Flatten layers, and add Densley connected layers for prediction
deep_cnn.add(keras.layers.Flatten())

# Dense layer with 32 nodes
deep_cnn.add(keras.layers.Dense(32, activation='relu'))

# Dense layer with 64 nodes
deep_cnn.add(keras.layers.Dense(64, activation='relu'))

# Dense layer with 96 nodes
deep_cnn.add(keras.layers.Dense(96, activation='relu'))

# Sigmoid output layer
deep_cnn.add(keras.layers.Dense(1, 'sigmoid'))


#Compile model
deep_cnn.compile(
    loss='binary_crossentropy',
    optimizer='sgd',
    # Adding additonal metrics for better monitoring of training.
    metrics=['acc', 'Recall', 'Precision']
    
)

# Fit Model to Training
deep_cnn_results = deep_cnn.fit_generator(train_data,
                              steps_per_epoch=100,
                              epochs=10,
                              validation_data=test_data)

**Conclusion**

I added additional metrics on this model for more insights into the results of the training proccess. As far as performance goes it's definetly an improvement from the last model in terms of validation accuracy.

Some other notes about the model:

- The model is still overfitting
- The validation accuracy is not conistently improving
- Validation recall is very high, ~97% of true positives were identified correctly. This is good, since we decided that, in context of our buisness problem, false negatives are more costly then false positives.
- Lets change the optimizer to see if this will improve the overfitting issues.

### **CNN model with added layers and Adam optimizer**

In [ ]:
# Set up for this CNN model is from this blog:  https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
cnn_adam = keras.Sequential()
# We defined a variable input_shape earlier, can use that here
cnn_adam.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape))
cnn_adam.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
cnn_adam.add(layers.MaxPooling2D((2, 2)))
cnn_adam.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
cnn_adam.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
cnn_adam.add(layers.MaxPooling2D((2, 2)))
cnn_adam.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
cnn_adam.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
cnn_adam.add(layers.MaxPooling2D((2, 2)))

# now, to get the proper output
cnn_adam.add(layers.Flatten())
cnn_adam.add(layers.Dense(128, activation='relu'))
cnn_adam.add(layers.Dense(1, activation='sigmoid'))

cnn_adam.compile(loss='binary_crossentropy',
            optimizer="adam",
            metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

In [ ]:
cnn_adam_results = cnn_adam.fit_generator(train_data,
                              steps_per_epoch=100,
                              epochs=10,
                              validation_data=test_data)

In [ ]:
visualize_results(cnn_adam_results)

**Conclusions**

This cnn models requires much improvement, because it is overfitting and as the training accuracy increases, the testing accuracy decreases, and as the training loss decreasesthe testing loss increases quite a lot. The double convolution layers before pooling may not be beneficial to increasing the model, so the convolution strategy will look more like the previous model in the next iteration.

Reading the keras documentation for adam optimizers, there was a note discussing how for some types of CNN models, the default value for the hyperparameter epsilon in adam (1e-7) may not be the best; they suggest trying bigger values such as 0.1 or 1. This was attempted in a notebook called "Brooke Image Classification," but it was not beneficial to the ultimate progression of models, so it is not included here.

## **CNN with Dropout Layers, Early Stopping, and More Training**

In [ ]:
# Create early stopping object
early_stopping = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=3),
    keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5')
]

# Create model
es_cnn = keras.Sequential()

# Adding first Conv2D and MaxPool layer, starting small and then growing larger.
es_cnn.add(keras.layers.Conv2D(32, (2, 2), activation='relu', input_shape=(150, 150, 3)))
es_cnn.add(keras.layers.MaxPool2D(2, 2))

# Second layer with 64 filters
es_cnn.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
es_cnn.add(keras.layers.MaxPool2D(3, 3))

# Third layer with 96 filters
es_cnn.add(keras.layers.Conv2D(96, (5, 5), activation='relu'))
es_cnn.add(keras.layers.MaxPool2D(5, 5))
# Flatten layers, and add Densley connected layers for prediction
es_cnn.add(keras.layers.Flatten())

# Dense layer with 32 nodes with dropout layer
es_cnn.add(keras.layers.Dense(32, activation='relu'))
es_cnn.add(keras.layers.Dropout(0.3))

# Dense layer with 64 nodes with dropout layer
es_cnn.add(keras.layers.Dense(64, activation='relu'))
es_cnn.add(keras.layers.Dropout(0.3))

# Dense layer with 96 nodes with dropout layer
es_cnn.add(keras.layers.Dense(96, activation='relu'))
es_cnn.add(keras.layers.Dropout(0.3))
# Sigmoid output layer
es_cnn.add(keras.layers.Dense(1, 'sigmoid'))

#Compile model
es_cnn.compile(
    loss='binary_crossentropy',
    optimizer='sgd',
    # Adding additonal metrics for better monitoring of training.
    metrics=['acc', 'Recall', 'Precision']
    
)

# Fit Model to Training
es_cnn_results = es_cnn.fit_generator(train_data,
                              steps_per_epoch=150,
                              epochs=25,
                              validation_data=test_data,
                              callbacks=early_stopping)



In [ ]:
visualize_results(es_cnn_results)

In [ ]:
oes_matrix(visualize_results)

**Conclusion**

Early stopping is working as intended, however, I've noticed that the first few epochs always have the same validation accuracy: 0.6250.

The model may be finding the local minimim instead of the global in these epochs. I'll try to tune the learning rate of my optimizer, and seeeing if that changes anything. I will also try to introduce class weights to the model, as that may help with the problem as well.

In [ ]:
#### # Create model
op_cnn = keras.Sequential()

# Adding first Conv2D and MaxPool layer, starting small and then growing larger.
op_cnn.add(keras.layers.Conv2D(32, (2, 2), activation='relu', input_shape=(150, 150, 3)))
op_cnn.add(keras.layers.MaxPool2D(2, 2))

# Second layer with 64 filters
op_cnn.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
op_cnn.add(keras.layers.MaxPool2D(3, 3))

# Third layer with 96 filters
op_cnn.add(keras.layers.Conv2D(96, (5, 5), activation='relu'))
op_cnn.add(keras.layers.MaxPool2D(5, 5))
# Flatten layers, and add Densley connected layers for prediction
op_cnn.add(keras.layers.Flatten())

# Dense layer with 32 nodes with dropout layer
op_cnn.add(keras.layers.Dense(32, activation='relu'))
op_cnn.add(keras.layers.Dropout(0.3))

# Dense layer with 64 nodes with dropout layer
op_cnn.add(keras.layers.Dense(64, activation='relu'))
op_cnn.add(keras.layers.Dropout(0.3))

# Dense layer with 96 nodes with dropout layer
op_cnn.add(keras.layers.Dense(96, activation='relu'))
op_cnn.add(keras.layers.Dropout(0.3))
# Sigmoid output layer
op_cnn.add(keras.layers.Dense(1, 'sigmoid'))

# Create early stopping object
op_early_stopping = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5')
]
# Create optimizer
optim = SGD(learning_rate=0.001, momentum=0.9, nesterov=True)

# Creating class weights
weights = {
    0: 2.88, # NORMAL
    1: 1.    # PNEM
}
#Compile model
op_cnn.compile(
    loss='binary_crossentropy',
    optimizer=optim,
    # Adding additonal metrics for better monitoring of training.
    metrics=['acc', 'Recall', 'Precision']
    
)

# Fit Model to Training
op_cnn_results = op_cnn.fit_generator(train_data,
                              class_weight=weights,
                              steps_per_epoch=50,
                              epochs=100,
                              validation_data=test_data,
                              callbacks=op_early_stopping)

In [ ]:
visualize_results(op_cnn_results)

In [ ]:
oes_matrix(op_cnn_results)

**Conclusion**

The early stopping worked great this time, and the changes to the optimizer, as well as adding class weights,has had a positive impact on the model. Validation accuracy is now sitting around ~87%. I'm going to add more layers and padding to the conv layers next, and see if that helps.